In [1]:
import glob

import numpy as np

from skimage.io import imread
from skimage.feature import hog

from sklearn.svm import LinearSVC
from skimage import exposure
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import time

import matplotlib.pyplot as plt
import os

from features import FeaturesRGB

Class mapping the name of the original and the augmented one

In [2]:
class LftToAuglft:
    def __init__(self, lft_filename, idx):
        self.lft_name = lft_filename
        self.self_idx = idx
        self.aug_lft_idx = []

HOG parameters, paths for dataset

In [3]:
hog_params = {
    'orientations': 11, # number of bins
    'pixels_per_cell': (8, 8), # normally (8,8), refers to 8x8 pixels are used to calculate hog
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}

ftrRGB = FeaturesRGB(hog_params)
lft_paths = glob.glob('../images/lateral/training/cropped/*.jpg')
auglft_paths = glob.glob('../images/lateral/training/augmented/*.jpg')

nonlft_paths = glob.glob('../images/non-lateral/training/*.jpg')


## Main

#### HOG for LFT

In [4]:
print("Load lateral flow test images, get HOG and add to the feature list ...")

lft_features, auglft_features, nonlft_features = [], [], []
lft_ftToAuglft = {}
lft_ftToAuglft_list = []
auglft_filename = []

count = 0

print("Loading and generating HOG for LFT...")
for path in lft_paths: 
    # Get file name
    f_split = os.path.basename(path)
    name_tmp = os.path.splitext(f_split)[0]
    
    lftToAuglft_obj = LftToAuglft(name_tmp, count)
    
    lft_ftToAuglft[name_tmp] = lftToAuglft_obj
    lft_ftToAuglft_list.append(lftToAuglft_obj)
    
    #lft_ftToAuglft.append(LftToAuglft(os.path.splitext(f_split)[0]))
    
    lft_img = imread(path) 

    hog_flatten = ftrRGB.cal_hog(lft_img, 0)
    lft_features.append(hog_flatten)
    
    
    count = count + 1
    #if count == 20:
        #break
        
print("Finish, files loaded: ", count)




Load lateral flow test images, get HOG and add to the feature list ...
Loading and generating HOG for LFT...
Finish, files loaded:  148


#### Get the filename for augmented LFT

In [ ]:
count = 0
auglft_filename = []

print("Loading files for augmented LFT...")
for path in auglft_paths: 
    # Get file name
    f_split = os.path.basename(path)
    name_tmp = os.path.splitext(f_split)[0]
    name_prefix_tmp = name_tmp.split('_')[0]

    
    
    if name_prefix_tmp in lft_ftToAuglft:
        lft_ftToAuglft[name_prefix_tmp].aug_lft_idx.append(len(auglft_filename))
    else:
        print("Cannot find the original image: ", name_prefix_tmp)
        break
        
        
    auglft_filename.append(name_tmp)
    
    count = count + 1

print("Finish, files loaded: ", count)  


for key, value in lft_ftToAuglft.items():    
    print(key, value.aug_lft_idx, value.self_idx)


#### HOG for augmented LFT

In [ ]:
count = 0
auglft_features = []

print("Loading and generating HOG for augmented LFT...")
for path in auglft_paths: 

    lft_img = imread(path) 
    
    hog_flatten = ftrRGB.cal_hog(lft_img, 0)
    auglft_features.append(hog_flatten)
    
    count += 1
    
    
print("Finish, files loaded: ", count)  

#### HOG for non-LFT

In [ ]:
print("Load non-lateral flow test images, get HOG and add to the feature list ...")

count = 0  
nonlft_features = []
print("Loading and generating HOG for non-LFT...")
  
for path in nonlft_paths: 
    nonlft_image = imread(path)

    hog_flatten = ftrRGB.cal_hog(nonlft_image, 0)
    
    nonlft_features.append(hog_flatten)
    
    count = count + 1
    #if count == 10:
        #break
        
print("Finish, files loaded: ", count)  


### Convert to numpy

In [ ]:

for i in range(0, len(nonlft_features)):
    print(i, len(nonlft_features[i]), nonlft_features[i])

In [ ]:
np_lft_features = np.asarray(lft_features)
np_auglft_features = np.asarray(auglft_features)
np_nonlft_features = np.asarray(nonlft_features)

In [ ]:
print("np_lft_features: ", np_lft_features.shape)
print("np_auglft_features: ", np_auglft_features.shape)
print("np_nonlft_features: ", np_nonlft_features.shape)

unscaled_x = np.vstack((np_lft_features, np_auglft_features, np_nonlft_features)).astype(np.float64)
print(unscaled_x.shape)

#print(unscaled_x)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)

print(x.shape)
#print(x)

In [ ]:
lft_cnt = np_lft_features.shape[0]
auglft_cnt = np_auglft_features.shape[0]
nonlft_cnt = np_nonlft_features.shape[0]



x_lft = x[:lft_cnt, :]
x_auglft = x[lft_cnt:auglft_cnt+lft_cnt, :] 
x_nonlft = x[auglft_cnt+lft_cnt:, :] 

print(x_lft.shape)
print(x_auglft.shape)
print(x_nonlft.shape)

#y = np.hstack((np.ones(lft_cnt), np.zeros(nonlft_cnt)))

In [ ]:
y = np.hstack((np.ones(lft_cnt + auglft_cnt), np.zeros(nonlft_cnt)))


print(" x shape: ", x.shape, " y shape: ", y.shape)

In [ ]:
print("Training classifier...")
t_start = time.time()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,
                                                    random_state =0)

svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Accuracy: ", np.round(accuracy, 4))

In [ ]:
print ("Saving models...")
import joblib

joblib.dump(svc, '../svc2.pkl')
joblib.dump(scaler, '../scaler2.pkl')

print("...Done")

In [ ]:
        """
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        
        ax[0].axis('off')
        ax[0].title.set_text('Original')
        #R_hog_image_rescaled = exposure.rescale_intensity(self.R_hog_image, in_range=(0, 10))
        #ax[0].imshow(R_hog_image_rescaled, cmap=plt.cm.gray)
        ax[0].imshow(self.RGB_img)
        
        ax[1].axis('off')
        ax[1].title.set_text('HOG (pixels_per_cell 8x8)')
        R_hog_image_rescaled = exposure.rescale_intensity(self.R_hog_image, in_range=(0, 10))
        ax[1].imshow(R_hog_image_rescaled, cmap=plt.cm.gray)
        
        #ax[2].axis('off')
        #B_hog_image_rescaled = exposure.rescale_intensity(self.B_hog_image, in_range=(0, 10))
        #ax[2].imshow(B_hog_image_rescaled, cmap=plt.cm.gray)  
        """